In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
from collections import defaultdict
from typing import Dict

import pandas as pd

from depsurf import Dep, DepKind, DepReport, IssueEnum
from utils import load_pkl, save_latex, rotate

data: Dict[str, Dict[Dep, DepReport]] = load_pkl("bcc")
data["Summary"] = {
    dep: report for deps in data.values() for dep, report in deps.items()
}

results = {}

KEY_NUMBER = "Count"

for prog_name, deps in data.items():
    if len(deps) == 0:
        print(f"Program {prog_name} has no dependencies")
        continue

    result = defaultdict(int)
    for dep, report in deps.items():
        kind = dep.kind
        result[(kind, KEY_NUMBER)] += 1

        num_absent = sum(
            1 for e in report.status.values() if IssueEnum.ABSENT in e.enums
        )
        result[(kind, IssueEnum.ABSENT)] += min(num_absent, 1)

        if kind != DepKind.STRUCT:
            num_delta = sum(1 for e in report.delta.values() if e.changes)
            result[(kind, IssueEnum.CHANGE)] += min(num_delta, 1)

        if kind == DepKind.FUNC:
            for issue in [
                IssueEnum.PARTIAL_INLINE,
                IssueEnum.FULL_INLINE,
                IssueEnum.RENAME,
                IssueEnum.DUPLICATE,
            ]:
                num_issue = sum(1 for e in report.status.values() if issue in e.enums)
                result[(kind, issue)] += min(num_issue, 1)

    has_issue = False
    for (kind, key), count in result.items():
        if key == KEY_NUMBER:
            continue
        if count != 0:
            has_issue = True
            break

    if not has_issue:
        prog_name = prog_name + r" \checkmark"

    results[prog_name] = result

df = pd.DataFrame(results).T.fillna(0).astype(int)

column_order = [
    DepKind.FUNC,
    DepKind.STRUCT,
    DepKind.FIELD,
    DepKind.TRACEPOINT,
    DepKind.SYSCALL,
]
df = df.reindex(sorted(df.columns, key=lambda x: x[0]), axis=1)


def rename_cols(col):
    kind, key = col
    if kind in (DepKind.STRUCT, DepKind.FIELD):
        key = {
            (DepKind.STRUCT, KEY_NUMBER): "Count S",
            (DepKind.FIELD, KEY_NUMBER): "Count F",
            (DepKind.STRUCT, IssueEnum.ABSENT): "Absent S",
            (DepKind.FIELD, IssueEnum.ABSENT): "Absent F",
            (DepKind.FIELD, IssueEnum.CHANGE): "Changed F",
        }[col]
        kind = "Struct / Field"
    if kind == DepKind.TRACEPOINT:
        kind = "Tracept"
    if kind == DepKind.SYSCALL:
        kind = "Syscall"
    return (kind, key)


df.columns = df.columns.map(rename_cols)
df = df.sort_values(by=df.columns.tolist(), ascending=False)
df_non_summary = df.loc[df.index != "Summary"]
df = pd.concat([df_non_summary, df.loc[["Summary"]]])
df.loc[r"\textbf{# tools}"] = df_non_summary.map(lambda x: min(x, 1)).sum(axis=0)
# df.loc["Pct. Tools"] = (df.loc["No. Tools"] / len(df_non_summary) * 100).astype(int)


df = df.replace(0, r"\color{lightgray}{-}")
df.columns = df.columns.map(lambda x: (x[0], rotate(x[1])))
latex = df.to_latex(
    multicolumn_format="c|",
    column_format="l|rrrrrrr|rr|rrr|rrr|rrr",
)
latex = latex.replace(r"Summary", r"\midrule" + "\n" + r"\textbf{# uniq deps}")

save_latex(latex, "bcc")

df

[          pkl.py:18 ] INFO: Loding bcc from /Users/szhong/Downloads/bpf-study/output/bcc.pkl
[        latex.py:49 ] INFO: Saved bcc to /Users/szhong/Downloads/bpf-study/paper/tabs/bcc.tex


Program bashreadline has no dependencies
Program funclatency has no dependencies
Program gethostlatency has no dependencies
Program javagc has no dependencies
Program ksnoop has no dependencies


Function                                                                                                                                                                                                                                         Struct / Field                                                                                                                                                                     Tracept                                                                                            Syscall                                                                   
                     \rotatebox[origin=r]{90}{Count} \rotatebox[origin=r]{90}{Absent} \rotatebox[origin=r]{90}{Changed} \rotatebox[origin=r]{90}{Par. Inline} \rotatebox[origin=r]{90}{Full Inline} \rotatebox[origin=r]{90}{Renamed} \rotatebox[origin=r]{90}{Duplicate} \rotatebox[origin=r]{90}{Count S} \rotatebox[origin=r]{90}{Absent S} \rotatebox[origin=r]{90}{Count F} \rotatebox[origin=r]{90}{Absent F} \rotatebox[origin=r]{90}{Changed F} \rotatebox[origin=r]{90}{Count} \rotatebox[origin=r]{90}{Absent} \rotatebox[origin=r]{90}{Changed} \rotatebox[origin=r]{90}{Count} \rotatebox[origin=r]{90}{Absent} \rotatebox[origin=r]{90}{Changed}
klockstat                                         14                                3              \color{lightgray}{-}                                     4                  \color{lightgray}{-}              \color{lightgray}{-}                \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}                \color{lightgray}{-}            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}
biotop                                             5                                2                                 2                                     2                                     3              \color{lightgray}{-}                \color{lightgray}{-}                                 3               \color{lightgray}{-}                                 7                                  2                                   1                               2                                2              \color{lightgray}{-}            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}
cachestat                                          5                                2                                 2                                     2                  \color{lightgray}{-}              \color{lightgray}{-}                \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}                \color{lightgray}{-}                               2                                2                                 1            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}
fsdist                                             5                                2                                 1                                     2                  \color{lightgray}{-}                                 2                \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}              \color{lightgray}{-}               \color{lightgray}{-}                \color{lightgray}{-}            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}            \color{lightgray}{-}             \color{lightgray}{-}              \color{lightgray}{-}
vfsstat                                            5             \color{lightgray}{-}                                 2                                     5                  \color{lightgray}{-}              \color{